In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
import gc
from sklearn.model_selection import KFold, train_test_split, ParameterGrid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import wilcoxon

import joblib

In [1]:
df_testes = pd.read_csv("df_reduzido_PCA.csv")

NameError: name 'pd' is not defined

In [3]:
df_reduced_percent = df_testes #Parcela do dataset que roda na minha maquina

#descomentar essa linha no treinamento na maquina que aguenta o tranco
#df_reduced_percent = df_testes 

x = df_reduced_percent.drop('label', axis = 1)
y = df_reduced_percent['label']

x1 = x.values
y1 = y.values

In [4]:
y.value_counts()

label
1    2746934
0    2746900
Name: count, dtype: int64

In [5]:
# Hiperparâmetros Random Forest (RF)
rf_best = RandomForestClassifier(
    n_estimators=10,          
    max_depth=20,              
    min_samples_split=10,      
    min_samples_leaf=5,        
    max_features='sqrt',      
    n_jobs=-1,                
    random_state=42,
    verbose=1              
)

#Arrays de metricas
test_scores_RF, precision_scores_RF, recall_scores_RF, f1_scores_RF, accs_RF, par_RF = [], [], [], [], [], []

kf = KFold(n_splits=10, shuffle=True, random_state=42)

scaler = StandardScaler()


In [8]:
for train_index, test_index in kf.split(x1):
    X_train, X_test, Y_train, Y_test = x1[train_index], x1[test_index], y1[train_index], y1[test_index]
    
    X_train_RF = scaler.fit_transform(X_train)
    X_test_RF = scaler.transform(X_test)  

    rf_best.fit(X_train_RF, Y_train)
    y_pred = rf_best.predict(X_test_RF)
    
    test_scores_RF.append(accuracy_score(Y_test, y_pred))
    precision_scores_RF.append(precision_score(Y_test, y_pred, average='weighted'))
    recall_scores_RF.append(recall_score(Y_test, y_pred, average='weighted'))
    f1_scores_RF.append(f1_score(Y_test, y_pred, average='weighted'))
  
    data = {
    "Modelo": ["RF"],
    "Acurácia": [np.mean(test_scores_RF)],
    "Precisão": [np.mean(precision_scores_RF)],
    "Recall": [np.mean(recall_scores_RF)],
    "F1 Score": [np.mean(f1_scores_RF)]
}
    
    joblib.dump(rf_best, 'intrusion_detection_rf.pkl')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   29.9s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   20.4s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent 

In [1]:
metrics = pd.DataFrame(data)
metrics.to_csv('metrics_model_rf.csv', index=False)

NameError: name 'pd' is not defined

In [11]:
metrics

,Modelo,Acurácia,Precisão,Recall,F1 Score
0,RF,0.970492,0.971369,0.970492,0.970478


In [ ]:
modelo = joblib.load('intrusion_detection_rf.pkl')
previsoes = modelo.predict(X_test)